In [70]:
import pandas as pd
import numpy as np
import pickle as p

In [71]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(21)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=100, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [72]:
# item_subset

[3718,
 7108,
 3821,
 2867,
 4410,
 1221,
 202,
 2256,
 3767,
 1225,
 1205,
 3823,
 1204,
 2887,
 1195,
 4977,
 7114,
 7113,
 16000,
 350,
 3717,
 7110,
 3733,
 1203,
 7078,
 31,
 5069,
 7158,
 5060,
 1504,
 1,
 16017,
 3727,
 5074,
 203,
 3738,
 7965,
 1152,
 3716,
 2257,
 4978,
 2581,
 5073,
 3771,
 3046,
 1231,
 5501,
 2501,
 16016,
 3723,
 7009,
 21,
 7128,
 7109,
 5662,
 5581,
 4272,
 22,
 4273,
 7159,
 102,
 7575,
 3601,
 4728,
 2731,
 3816,
 3742,
 3772,
 2721,
 6750,
 2521,
 2736,
 7041,
 3737,
 4411,
 16018,
 7129,
 2258,
 3045,
 5690,
 2313,
 3721,
 281,
 5068,
 1615,
 5469,
 4976,
 3732,
 2551,
 3817,
 200,
 7080,
 284,
 4267,
 7967,
 5771,
 3766,
 3773,
 204,
 3731]

### `/train`

In [74]:
import sys
sys.getsizeof(data_subset)

245684504

In [75]:
data_subset.to_parquet('data_subset.parquet')

In [73]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'

data = data_subset.to_json()

files = {'data':open('data_subset.parquet', 'rb')}

payload = {'cv_acc': False,
           'project_id': 5
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers, timeout=1200, files=files)

files['data'].close()

KeyboardInterrupt: 

In [ ]:
r.json()['result']

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [58]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'1': 4.65,
 '31': 2.55,
 '202': 2.1,
 '203': 4.9,
 '350': 2.65,
 '1152': 4.2,
 '1195': 3.55,
 '1203': 3.45,
 '1204': 3.85,
 '1205': 4.25,
 '1221': 3.75,
 '1225': 2.55,
 '1231': 4.25,
 '1504': 4.85,
 '2256': 8.25,
 '2257': 11.8,
 '2501': 2.75,
 '2581': 5.1,
 '2867': 5.8,
 '2887': 8.0,
 '3046': 3.15,
 '3716': 6.2,
 '3717': 6.9,
 '3718': 7.6,
 '3723': 11.65,
 '3727': 10.95,
 '3733': 12.7,
 '3738': 10.25,
 '3767': 10.25,
 '3771': 9.2,
 '3821': 11.25,
 '3823': 12.65,
 '4410': 4.7,
 '4977': 12.3,
 '4978': 13.0,
 '5060': 5.2,
 '5069': 10.3,
 '5073': 3.95,
 '5074': 7.0,
 '5501': 3.5,
 '7078': 4.25,
 '7108': 3.7,
 '7110': 4.6,
 '7113': 3.75,
 '7114': 4.5,
 '7158': 5.25,
 '7965': 3.6,
 '16000': 7.25,
 '16016': 8.75,
 '16017': 12.0}

In [59]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 4}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)

r = requests.post(url, data=payload, headers=headers)
print(r.status_code)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

200


,Estimated,Actual
Qty_1,590.0,632.0
Qty_1152,117.0,103.0
Qty_1195,164.0,204.0
Qty_1203,186.0,177.0
Qty_1204,255.0,230.0
Qty_1205,225.0,262.0
Qty_1221,215.0,238.0
Qty_1225,27.0,32.0
Qty_1231,15.0,12.0
Qty_1504,586.0,590.0
